In [4]:
import os
import pandas as pd
from tqdm import tqdm
from os.path import isfile, join
from os import listdir
import time 
import re
import json
from datetime import datetime, date
import string

global fechas_contrato
with open('contratos_fechas.json', 'r', encoding='utf-8') as f:
    fechas_contrato = json.load(f, encoding='utf-8')

# 1. Planilla_buscador
# 2. Planilla_perfil (esto se hace en el 2_planilla_tra....py )
# 3. Visitantes_buscador.csv  (presidencial/congreso + ministerios)
# 4. Visitantes_perfil (presidencial/congreso + ministerios)
# 5. Proveedores_perfil (se hace en 8_provee....py)
# 6. Visitantes_empresarios (se hace en 10_encontra....py)
# 7. Proveedores_buscador

def obtener_files(string):
    directorio = os.getcwd()
    directorio_files = directorio + "/" + string
    onlyfiles = [f for f in listdir(directorio_files) if (isfile(join(directorio_files, f))) and (f[-4:] == ".csv" ) ]
    return onlyfiles

def get_fecha_contrato(texto):
    global fechas_contrato
    try:
        n_contrato = re.findall(r'\d+',texto)[-1]
        fecha = fechas_contrato[n_contrato]
        fecha = datetime.strptime(fecha, '%Y-%m-%d %H:%M:%S').date()
        return fecha
    except:
        return None

def obtener_df_de_firestore_planilla(string = "planilla_mensual", 
                nombre_guardar = "planilla_buscador"):
    onlyfiles = obtener_files("planilla_mensual")
    df_firestore = pd.DataFrame()
    for i in tqdm(onlyfiles, desc = "Downloading " + string):
        mensual_df = pd.read_csv(os.getcwd() + "/" + string + "/" + i, low_memory= False)
        if len(mensual_df) > 0:
            df_firestore = pd.concat([df_firestore, mensual_df])
    df_firestore.index = list(range(len(df_firestore)))
    df_firestore.index.name = "registro"
    df_firestore.drop(["Unnamed: 0"], axis = 1, inplace= True)
    columnas_ordenadas = ['VC_PERSONAL_NOMBRES',  'VC_PERSONAL_PATERNO',  'VC_PERSONAL_MATERNO',
    'ENTIDAD', 'VC_PERSONAL_DEPENDENCIA',  'VC_PERSONAL_REGIMEN_LABORAL',
    'VC_PERSONAL_CARGO', 'VC_PERSONAL_RUC_ENTIDAD', 'PK_ID_PERSONAL',
    'FEC_REG', 'IN_PERSONAL_ANNO', 'IN_PERSONAL_MES', 'MO_PERSONAL_TOTAL',
    'MO_PERSONAL_HONORARIOS', 'MO_PERSONAL_REMUNERACIONES', 'MO_PERSONAL_GRATIFICACION',
    'MO_PERSONAL_INCENTIVO', 'MO_PERSONAL_OTROS_BENEFICIOS', 'VC_PERSONAL_OBSERVACIONES']
    df_firestore = df_firestore[columnas_ordenadas]
    df_firestore.to_csv(os.getcwd() + "/producto/" + nombre_guardar + ".csv")
    
    #return df_firestore[columnas_ordenadas]
    
def obtener_df_de_firestore_visitas(string):
    onlyfiles = obtener_files(string)
    df_firestore = pd.DataFrame()
    for i in tqdm(onlyfiles, desc = "Downloading " + string):
        mensual_df = pd.read_csv(os.getcwd() + '/' + string + '/' + i, low_memory = False)
        if len(mensual_df) > 0:
            df_firestore = pd.concat([df_firestore, mensual_df])
    df_firestore.drop(['Unnamed: 0'], axis = 1, inplace= True)
    df_firestore.dropna(subset=['Visitante'], inplace= True)
    df_firestore.index = list(range(len(df_firestore)))
    return df_firestore

def obtener_visitantes_todos():
    df_visitas_pre_con = obtener_df_de_firestore_visitas("visitas_mensuales")
    df_visitas_min = obtener_df_de_firestore_visitas("visitas_ministerios_mensual")
    df_visitas_todos = pd.concat([df_visitas_pre_con , df_visitas_min])
    #del df_visitas_pre_con, df_visitas_min
    df_visitas_todos.index = list(range(len(df_visitas_todos)))
    columnas_ordenadas = ['fecha', 'Hora_Ingreso', "Hora_Salida", "Motivo",
    "Visitante", "Tipo_Documento", "N_Documento", "Institucion", "Visitado",
    "entidad", "Cargo", "Oficina", "Observacion"]
    df_visitas_todos = df_visitas_todos[columnas_ordenadas]
    df_visitas_todos.to_csv(os.getcwd() + '/producto/visitantes_buscador.csv')

def obtener_visitantes_transform_todos():
    df_visitante_trans_pre_con = pd.read_csv(os.getcwd() + "/visitas_transform/visitantes_transformed.csv")
    df_visitante_trans_min = pd.read_csv(os.getcwd() + "/visitas_ministerios_transform/visitantes_transformed.csv")
    df_visitante_trans_pre_con.sort_values(by=['id'], inplace = True)
    len_trans_pre_con = len(df_visitante_trans_pre_con)
    df_visitante_trans_pre_con.index = list(range(len_trans_pre_con))
    df_visitante_trans_min.sort_values(by=['id'], inplace=True)
    len_trans_min = len(df_visitante_trans_min)
    df_visitante_trans_min.index = list(range(len_trans_pre_con + 1, len_trans_pre_con + len_trans_min + 1 ))
    df_visitante_trans_min['id'] += len_trans_pre_con + 1
    df_visitas_transform_todos = pd.concat([df_visitante_trans_pre_con, df_visitante_trans_min])
    df_visitas_transform_todos.to_csv(os.getcwd() + '/producto/visitantes_perfil.csv')
    return df_visitas_transform_todos

def print_time(seconds, message):
    seconds = int(seconds)
    horas = seconds//3600
    minutos = seconds//60 - (horas*60)
    segundos = seconds - horas*3600 - minutos*60
    print(u"Takes: {} hours, {} minutes, {} seconds {}".format(horas, minutos, segundos, message))

def get_fecha(x, s = '-'):
    if (x != None) or (x!=''):
        #print(datetime.strptime(x[:10], '%Y-%m-%d').date())
        try:
            return datetime.strptime(x[:10], '%Y'+ s + '%m' + s + '%d').date()
        except:
            return None
    else:
        return None

def get_fecha_2(x, s = '-'):
    if (x != None) or (x!=''):
        #print(datetime.strptime(x[:10], '%Y-%m-%d').date())
        try:
            return datetime.strptime(x, '%d'+ s + '%m' + s + '%Y').date()
        except:
            return None
    else:
        return None

def obtain_months_past(x):
    if x[1] is not None:
        return (x[1].year - x[0].year) * 12 + (x[1].month - x[0].month)
    else:
        return 10000
        
def get_numb(x):
    return re.findall(r'\d+', x)[0]

def clean_text(text):
    tildes = ['á', 'é', 'í', 'ó', 'ú']
    normal = ['a', 'e', 'i', 'o', 'u']
    text = text.lower()
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ' , text)
    for j in range(len(tildes)):
        if tildes[j] in text:
            text = text.replace(tildes[j], normal[j])
    text = " ".join(text.split())
    text = text.strip()
    if (text[-2:] == ' i') | (text[-2:] == ' a' ) | (text[-2:] == ' e' ) | (text[-2:] == ' o' ):
        text = text[:-2]
    return text

def get_df(item, ruc, rucs):
    # 1. Seace
    df_seace = pd.DataFrame(item['experiencia_seace'])
    if len(df_seace) > 0:
        df_seace['fecha'] = df_seace['fecProgTerm'].apply(lambda x: get_fecha(x))
        df_seace['ruc'] = ruc
        df_seace['Razon_social'] = rucs[ruc]
    # 2. Organos
    df_organos= pd.DataFrame(item['organosAdm'])
    if len(df_organos) > 0:
        df_organos['ruc'] = ruc
        df_organos['Razon_social'] = rucs[ruc]
    # 3. Proveedor
    df_proveedor = pd.DataFrame.from_dict([item['proveedor']])

    # 4. Representación
    df_repre = pd.DataFrame(item['representantes'])
    if len(df_repre) > 0:
        df_repre['ruc'] = ruc
        df_repre['Razon_social'] = rucs[ruc]

    # 5. Sanciones
    df_sanciones = pd.DataFrame(item['sanciones'])
    if len(df_sanciones) > 0:
        df_sanciones['Razon_social'] = rucs[ruc]

    # 6. Socios
    df_socios = pd.DataFrame(item['socios'])

    if len(df_socios) > 0:
        df_socios['ruc'] = ruc
        df_socios['Razon_social'] = rucs[ruc]

    return df_seace, df_organos, df_proveedor, df_repre, df_sanciones, df_socios

def getting_proveedores_info():
    with open('rucs_proveedores_2021_2022.json', 'r', encoding='utf-8') as f:
        rucs = json.load(f, encoding='utf-8')
    f = open(os.getcwd() + '/proveedores/proveedores_load.txt', 'r')
    #lines = f.readlines()
    # Cum = dataframe total
    df_seace_cum = pd.DataFrame()
    df_organos_cum = pd.DataFrame()
    df_proveedor_cum = pd.DataFrame()
    df_repre_cum = pd.DataFrame()
    df_sanciones_cum = pd.DataFrame()
    df_socios_cum =pd.DataFrame()
    # aux = chunks
    df_seace_aux = pd.DataFrame()
    df_organos_aux = pd.DataFrame()
    df_proveedor_aux = pd.DataFrame()
    df_repre_aux = pd.DataFrame()
    df_sanciones_aux = pd.DataFrame()
    df_socios_aux = pd.DataFrame()
    cont=0
    with open(os.getcwd() + '/proveedores/proveedores_load.txt', 'r') as input_file:
        lines = input_file.readlines()
        #while line != "":
        for line in tqdm(lines, desc="loading proveedores"):
            if cont % 150 == 0:
                #print(cont, end = "\r")
                df_seace_cum = pd.concat([df_seace_cum, df_seace_aux])
                df_organos_cum = pd.concat([df_organos_cum, df_organos_aux])
                df_proveedor_cum = pd.concat([df_proveedor_cum, df_proveedor_aux])
                df_repre_cum = pd.concat([df_repre_cum,df_repre_aux])
                df_sanciones_cum = pd.concat([df_sanciones_cum, df_sanciones_aux])
                df_socios_cum = pd.concat([df_socios_cum,df_socios_aux])
                
                df_seace_aux = pd.DataFrame()
                df_organos_aux = pd.DataFrame()
                df_proveedor_aux = pd.DataFrame()
                df_repre_aux = pd.DataFrame()
                df_sanciones_aux = pd.DataFrame()
                df_socios_aux = pd.DataFrame()      
            try:
                linea = line[:-1]
                data = {}
                item = json.loads(linea)
                ruc = item['proveedor']['numeroRuc']
                df_seace, df_organos, df_proveedor, df_repre, df_sanciones, df_socios = get_df(item, ruc, rucs)
                df_seace_aux = pd.concat([df_seace_aux, df_seace])
                df_organos_aux = pd.concat([df_organos_aux, df_organos])
                df_proveedor_aux = pd.concat([df_proveedor_aux, df_proveedor])
                df_repre_aux = pd.concat([df_repre_aux,df_repre])
                df_sanciones_aux = pd.concat([df_sanciones_aux, df_sanciones])
                df_socios_aux = pd.concat([df_socios_aux,df_socios])
                cont += 1
            except:
                cont += 1
            line = input_file.readline()

    # Seace
    df_seace_cum['fecha'] = df_seace_cum['codContProv'].apply(lambda x : get_fecha_contrato(x))
    df_seace_cum.drop(['fecProgTerm', 'montoOrigenTexto'], axis = 1 , inplace = True)
    columnas_ordenadas = ['fecha', 'ruc', 'Razon_social', 'nomEntCont', 'desContProv', 'montoOrigen',
                    'desEstContProv', 'desCatObj2', 'codContProv']
    df_seace_cum = df_seace_cum[columnas_ordenadas]
    df_seace_cum.index = list(range(len(df_seace_cum)))
    df_seace_cum.to_csv(os.getcwd() + '/producto/proveedores_perfil_seace.csv',encoding= 'utf-8-sig')
    del df_seace_cum

    # Organos
    columnas_ordenadas = ['ruc', 'Razon_social','apellidosNomb', 'siglaDocIde', 'nroDocumento', 'descCargo',
                        'descTipoOrgano', 'fechaIngreso']
    df_organos_cum.loc[:, 'apellidosNomb'] = df_organos_cum['apellidosNomb'].apply(lambda x: clean_text(x))
    df_organos_cum = df_organos_cum[columnas_ordenadas]
    df_organos_cum.index = list(range(len(df_organos_cum)))
    df_organos_cum.to_csv(os.getcwd() + '/producto/proveedores_perfil_org_administrativos.csv',encoding= 'utf-8-sig')
    del df_organos_cum

    # Info geográfica
    columnas_ordenadas = ['numeroRuc','razonSocial','tipoEmpresa','departamento','distrito','provincia','estado','condicion',
    'codigoRegistro','clcTexto', 'cmcTexto']
    df_proveedor_cum = df_proveedor_cum[columnas_ordenadas]
    df_proveedor_cum.index = list(range(len(df_proveedor_cum)))
    df_proveedor_cum.rename(columns={"numeroRuc":"ruc"}, inplace = True)
    df_proveedor_cum.to_csv(os.getcwd() + '/producto/proveedores_perfil_info_geo.csv',encoding= 'utf-8-sig')
    del df_proveedor_cum

    # Representates
    df_repre_cum['apellidosNomb'] = df_repre_cum['razonSocial'].apply(lambda x: clean_text(x))
    columnas_ordenadas = ['ruc', 'Razon_social', 'apellidosNomb', 'siglaDocIde', 'nroDocumento']
    df_repre_cum = df_repre_cum[columnas_ordenadas]
    df_repre_cum.index = list(range(len(df_repre_cum)))
    df_repre_cum.to_csv(os.getcwd() + '/producto/proveedores_perfil_representantes.csv',encoding= 'utf-8-sig')
    del df_repre_cum

    # Sanciones
    df_sanciones_cum['fechaFin_'] = df_sanciones_cum['fechaFin'].apply(lambda x: get_fecha_2(x , '/'))
    df_sanciones_cum['fechaIni_'] = df_sanciones_cum['fechaIni'].apply(lambda x: get_fecha_2(x, '/'))
    df_sanciones_cum['fechaResol'] = df_sanciones_cum['fechaResol'].apply(lambda x: get_fecha_2(x, '/'))
    df_sanciones_cum['meses_sancionado'] = df_sanciones_cum[['fechaIni_', 'fechaFin_']].apply(obtain_months_past, axis = 1)
    df_sanciones_cum['idsCausales'] = df_sanciones_cum['idsCausales'].apply(lambda x: re.findall(r'\d+', x)[0])
    df_sanciones_cum['motivos'] = df_sanciones_cum['motivos'].apply(lambda x: x[0])
    df_sanciones_cum['motivos'] = df_sanciones_cum['motivos'].apply(lambda x: clean_text(x))
    df_sanciones_cum.drop(['fechaFin', 'fechaIni', 'periodo'], axis = 1, inplace = True)
    columnas_ordenadas = ['numeroRuc','Razon_social','descripcion','idsCausales','motivos','fechaIni_','fechaFin_',
    'meses_sancionado','vigente','montoTexto','nroRes','fechaResol','idNumInhab']
    df_sanciones_cum = df_sanciones_cum[columnas_ordenadas]
    df_sanciones_cum.index = list(range(len(df_sanciones_cum)))
    df_sanciones_cum.rename(columns={"numeroRuc":"ruc"}, inplace = True)
    df_sanciones_cum.to_csv(os.getcwd() + '/producto/proveedores_perfil_sanciones.csv',encoding= 'utf-8-sig')
    del df_sanciones_cum

    # Socios
    columnas_ordenadas = ['ruc', 'Razon_social', 'razonSocial','siglaDocIde','nroDocumento',
                        'fechaIngreso', 'porcentajeAcciones']
    df_socios_cum = df_socios_cum[columnas_ordenadas]
    df_socios_cum.index = list(range(len(df_socios_cum)))
    df_socios_cum.rename(columns={"razonSocial":"Razon_social_socio"}, inplace = True)
    df_socios_cum.to_csv(os.getcwd() + "/producto/proveedores_perfil_socios.csv",encoding= 'utf-8-sig')
    del df_socios_cum


In [5]:
getting_proveedores_info()

loading proveedores: 100%|██████████| 24333/24333 [12:01<00:00, 33.73it/s]  


In [15]:
with open('rucs_proveedores_2021_2022.json', 'r', encoding='utf-8') as f:
    rucs = json.load(f, encoding='utf-8')
f = open(os.getcwd() + '/proveedores/proveedores_load.txt', 'r')
lines = f.readline()
# Cum = dataframe total
df_sanciones_cum = pd.DataFrame()
# aux = chunks
df_sanciones_aux = pd.DataFrame()
cont=0

#with open("datos_estudiante.txt","r") as input_file:
#    linea = input_file.readline()
#    while linea !="":       #para separar linea por linea
#        lista=linea.split()  #El nombre y las notas de cada linea se guarda en una lista donde solo aparece notas
#        s=0
#        for i in range(1,len(lista)):   #los numeros están desde la posición 1, hasta el final (len)
#            s=s+ int(lista[i])  #Se suma todas las notas
#        prom=s/(len(lista)-1)   #Se resta 1 porque en la lista, el primer valor es el nombre y yo solo quiero notas
#        print(lista[0],prom)
#        linea = input_file.readline()   #Una vez que saca el promedio de una persona, pasa a la siguiente línea
    

with open(os.getcwd() + '/proveedores/proveedores_load.txt', 'r') as input_file:
    line = input_file.readline()
    while line != "":
        #for line in tqdm(lines, desc= 'Creando tablas con toda la info de proveedores '):
        if cont % 150 == 0:
            df_sanciones_cum = pd.concat([df_sanciones_cum, df_sanciones_aux])
            df_sanciones_aux = pd.DataFrame()    
            print(cont, end = "\r")
        try:
            linea = line[:-1]
            data = {}
            item = json.loads(linea)
            ruc = item['proveedor']['numeroRuc']
            df_seace, df_organos, df_proveedor, df_repre, df_sanciones, df_socios = get_df(item, ruc, rucs)
            df_sanciones_aux = pd.concat([df_sanciones_aux, df_sanciones])
            cont += 1
        except:
            cont += 1
        line = input_file.readline()


# Sanciones
df_sanciones_cum['fechaFin_'] = df_sanciones_cum['fechaFin'].apply(lambda x: get_fecha_2(x , '/'))
df_sanciones_cum['fechaIni_'] = df_sanciones_cum['fechaIni'].apply(lambda x: get_fecha_2(x, '/'))
df_sanciones_cum['fechaResol'] = df_sanciones_cum['fechaResol'].apply(lambda x: get_fecha_2(x, '/'))
df_sanciones_cum['meses_sancionado'] = df_sanciones_cum[['fechaIni_', 'fechaFin_']].apply(obtain_months_past, axis = 1)
df_sanciones_cum['idsCausales'] = df_sanciones_cum['idsCausales'].apply(lambda x: re.findall(r'\d+', x)[0])
df_sanciones_cum['motivos'] = df_sanciones_cum['motivos'].apply(lambda x: x[0])
df_sanciones_cum['motivos'] = df_sanciones_cum['motivos'].apply(lambda x: clean_text(x))

#df_sanciones_cum.drop(['fechaFin', 'fechaIni', 'periodo'], axis = 1, inplace = True)
#columnas_ordenadas = ['numeroRuc','Razon_social','descripcion','idsCausales','motivos','fechaIni_','fechaFin_',
#'meses_sancionado','vigente','montoTexto','nroRes','fechaResol','idNumInhab']
#df_sanciones_cum = df_sanciones_cum[columnas_ordenadas]
df_sanciones_cum.index = list(range(len(df_sanciones_cum)))
df_sanciones_cum.rename(columns={"numeroRuc":"ruc"}, inplace = True)
df_sanciones_cum.to_csv(os.getcwd() + '/producto/proveedores_perfil_sanciones.csv')
#del df_sanciones_cum



In [13]:
df_sanciones_cum.head()

,descripcion,fechaFin,fechaIni,fechaResol,idNumInhab,idsCausales,montoTexto,motivos,nroRes,ruc,periodo,vigente,Razon_social,fechaFin_,fechaIni_,meses_sancionado
0,MULTA,08/12/2021,08/06/2021,2021-05-20,0020900,237,10643.20,b) Incumplir injustificadamente con su obligac...,1195-2021-TCE-S3,20600190301,6 MESES,False,SOLUCIONES INNOVADORAS J&B S.A.C.,2021-12-08,2021-06-08,6
1,MULTA,10/03/2020,10/10/2019,2019-09-20,0018106,207,16710.00,Incumplir con su obligación de perfeccionar el...,2648-2019-TCE-S4,20600190301,5 MESES,False,SOLUCIONES INNOVADORAS J&B S.A.C.,2020-03-10,2019-10-10,5
2,TEMPORAL,15/07/2022,15/03/2022,2022-03-07,0023302,238,,c) Contratar con el Estado estando impedido co...,782-2022-TCE-S1,20129383659,4 MESES,True,COMERCIAL GUILLERMO EIRLTDA,2022-07-15,2022-03-15,4
3,TEMPORAL,09/04/2011,10/02/2010,2010-01-26,0003612,24,,RESOLUCION DE CONTRATO,146-2010-TC-S3,20531607784,CATORCE MESES,False,SERVICIOS DEL ACERO E.I.R.L.,2011-04-09,2010-02-10,14
4,TEMPORAL,13/12/2011,14/07/2010,2010-06-25,0004148,08,,DOCUMENTOS FALSOS,1271-2010-TC,20528064028,DIECISIETE MESES,False,MULTISERVICIOS EL DIAMANTE SOCIEDAD ANONIMA CE...,2011-12-13,2010-07-14,17


In [12]:
for i in df_sanciones_cum['motivos'].unique():
    print(i , end = "*")

b) Incumplir injustificadamente con su obligación de perfeccionar el contrato o de formalizar Acuerdos Marco.*Incumplir con su obligación de perfeccionar el contrato o de formalizar Acuerdos Marco.*c) Contratar con el Estado estando impedido conforme a Ley.*RESOLUCION DE CONTRATO*DOCUMENTOS FALSOS*Presentar información inexacta a las Entidades, al Tribunal de Contrataciones del Estado o al Registro Nacional de Proveedores (RNP), siempre que esté relacionada con el cumplimiento de un requerimiento o factor de evaluación que le represente una ventaja o beneficio en el procedimiento de selección o en la ejecución contractual.*Desistirse o retirar injustificadamente su propuesta.*Ocasionar que la Entidad resuelva el contrato, incluido Acuerdos Marco, siempre que dicha resolución haya quedado consentida o firme en vía conciliatoria o arbitral.*NO SUSCRIPCION DE CONTRATO*a) Desistirse o retirar injustificadamente su oferta.*Den lugar a la resolución del contrato o de servicios por causal atr

In [2]:
if __name__ == '__main__':
    t0 = time.time()
    print("********************         PRODUCTOS        ********************\n\n")
    #obtener_df_de_firestore_planilla()      #1
    #obtener_visitantes_todos()              #3
    #obtener_visitantes_transform_todos()    #4
    getting_proveedores_info()              #8
    t1 = time.time()
    print_time(t1 - t0, " to generate the .csvs.")

********************         PRODUCTOS        ********************




Creando tablas con toda la info de proveedores : 100%|██████████| 24333/24333 [07:35<00:00, 53.40it/s] 


Takes: 0 hours, 8 minutes, 9 seconds  to generate the .csvs.


In [3]:
import pandas as pd
import os

In [4]:
proveedores = pd.read_csv(os.getcwd() + '/producto/proveedores_perfil.csv')

In [7]:
proveedores['tipoEmpresa'].unique()

array(['persona natural con negocio', 'persona natural sin negocio',
       'sociedad anonima cerrada', 'sociedad anonima',
       'empresa individual de resp ltda', 'soc com respons ltda', nan,
       'asociacion', 'sucursales o ag de emp extranj', 'fundacion',
       'sociedad civil', 'cooperativas sais caps',
       'comunidad campesina nativa', 'univers centros educat y cult',
       'sociedad irregular', 'sociedad conyugal sin negocio',
       'instituciones publicas', 'sociedad anonima abierta',
       'empresa de propiedad social', 'sucesion indivisa con negocio',
       'cafaes y subcafaes', 'instituciones religiosas',
       'empresa estatal de derecho privado',
       'sucesion indivisa sin negocio', 'sociedad conyugal con negocio',
       'sociedad en comandita por acciones'], dtype=object)